In [ ]:
%pylab inline

#### Load constants ####
import scipy.constants as sc

import matplotlib as mpl
import glob
import os

import sys

import adios2

# OVERWRITE DEFAULT PLOTTING PARAMETERS
params = {
    'font.family' : 'sans-serif',
    'font.sans-serif' : ['DejaVu Sans', 'Arial', 'Verdana', 'Liberation Sans'],
    'mathtext.default' : 'regular',
    'mathtext.rm' : 'sans',
    'font.size' : 18,
    'legend.fontsize' : 14,
    'axes.labelsize' : 18,
    'axes.titlesize' : 16,
    'lines.linewidth' : 3.0,
    'legend.frameon' : False,
    'legend.numpoints': 1,
}

matplotlib.rcParams.update(params)

In [ ]:
#path = ("/gpfs/alpine/csc434/scratch/ksteinig/2021-06_Tweac-cavity-search/001_phi-3.5_base-2.5_a-3_gpu-16/simOutput/")
#path = ("/gpfs/alpine/csc434/scratch/ksteinig/2021-06_Tweac-cavity-search/002_phi-5_w-5_a0-3.5_dens-2.5_gpu-16/simOutput/")
#path = ("/gpfs/alpine/csc434/scratch/ksteinig/2021-06_Tweac-cavity-search/003_phi-5_w-1.2_a0-3.5_dens-2.5_25ppc_gpu-16/simOutput/")
path = ("/gpfs/alpine/csc434/scratch/ksteinig/2021-06_Tweac-cavity-search/004_phi-5_w-1.2_a0-3.5_dens-2.5_25ppc_gpu-1-8-2/simOutput/")

files = glob.glob(path + "/openPMD/" + "*.bp")
files.sort(key=os.path.getmtime)

print("Number of files = %i"%(len(files)))

print(files)

# Grid parameters

In [ ]:
f = adios2.open(files[0], "r")

print("index order:", f.read_attribute_string("/data/0/fields/B/axisLabels"))

Nz, Ny, Nx = array(f.available_variables().get("/data/0/fields/B/x")['Shape'].split(', '), dtype=int)
print("Nz = %i, Ny = %i, Nx = %i"%(Nz,Ny,Nx))

unit_length_SI = f.read_attribute("/data/0/unit_length")
Dy_SI = f.read_attribute("/data/0/cell_height") * unit_length_SI
Dx_SI = f.read_attribute("/data/0/cell_width") * unit_length_SI
Dz_SI = Dx_SI

unit_time_SI = f.read_attribute("/data/0/unit_time")
Dt_SI = f.read_attribute("/data/0/dt") * unit_time_SI
print("Δt = %es"%(Dt_SI))

f.close()

# Particle count and energy stability

In [ ]:
nt, ne = loadtxt(path + '/e_macroParticlesCount.dat', usecols=(0,1), unpack=True)
nt_e, e_tot = loadtxt(path + '/e_energyHistogram_all.dat', usecols=(0,1), unpack=True)

In [ ]:
figure(figsize=(16,6))

subplot(1,2,1)

plot(nt[1:], ne[1:]/ne[1])
xlabel("time step")
#xticks(nt[::500])
title("#particles")
grid()


subplot(1,2,2)

plot(nt_e[1:-1:2], e_tot[1:-1:2]/sc.e/ne[1:-1:2] * 1.e-6)
xlabel("time step")
#xticks(nt[::500])
title("average single macro e kin. energy [mc^2]")
grid()

tight_layout()
show()
close()

# Charge distribution

In [ ]:
for i in arange(len(files)): #array([10]):
    
    #Load data
    f = adios2.open(files[i], "r")
    
    #extract time step
    t_i = int((files[i].rpartition('_')[2]).rpartition('.')[0])
    #print(t_i)
    
    unit_chargeDensity = f.read_attribute("/data/%i/fields/e_all_chargeDensity/unitSI"%(t_i))[0]
    e_numberDensity = f.read("/data/%i/fields/e_all_chargeDensity"%(t_i))*(unit_chargeDensity/(-sc.e))
    
    f.close()
    
    figure(figsize=(16,6))
    
    subplot(1,2,1)
    imshow(abs(e_numberDensity[:, :, Nx//2])*Dx_SI*Dy_SI*Dz_SI, cmap = mpl.cm.cubehelix_r
           , origin='lower', interpolation='none', aspect='auto'
           #, norm=matplotlib.colors.LogNorm()#vmin = 1.e3, vmax = 2.e5)
           , vmin = 50000, vmax = 200000
    )
    
    colorbar()
    
    title("Real electron number per cell\n@t = %iDt"%(t_i))
    xlabel("y [cells]")
    ylabel("z [cells]")
    
    
    subplot(1,2,2)
    
    imshow(abs(e_numberDensity[Nz//2, :, :]).T*Dx_SI*Dy_SI*Dz_SI, cmap = mpl.cm.cubehelix_r
           , origin='lower', interpolation='none', aspect='auto'
           #, extent = (0., Ny, -30.*Dx_SI*1.e6, 29.*Dx_SI*1.e6)
           #, norm=matplotlib.colors.LogNorm()#vmin = 1.e3, vmax = 2.e5)
           , vmin = 50000, vmax = 200000
    )
    
    colorbar()
    
    xlabel("y [cells]")
    ylabel("x [cells]")
    
    tight_layout()
    
    show()
    
    close()


# Electric field distribution

In [ ]:
for i in arange(len(files)): #array([5]): #arange(len(files)):
    
    #Load data
    f = adios2.open(files[i], "r")
    
    #extract time step
    t_i = int((files[i].rpartition('_')[2]).rpartition('.')[0])
    #print(t_i)
    
    unit_fieldE = f.read_attribute("/data/%i/fields/E/x/unitSI"%(t_i))[0]
    Ex = f.read("/data/%i/fields/E/x"%(t_i)) * unit_fieldE
    Ey = f.read("/data/%i/fields/E/y"%(t_i)) * unit_fieldE
    Ez = f.read("/data/%i/fields/E/z"%(t_i)) * unit_fieldE
    
    unit_fieldB = f.read_attribute("/data/%i/fields/B/z/unitSI"%(t_i))[0]
    Bx = f.read("/data/%i/fields/B/x"%(t_i)) * unit_fieldB
    By = f.read("/data/%i/fields/B/y"%(t_i)) * unit_fieldB
    Bz = f.read("/data/%i/fields/B/z"%(t_i)) * unit_fieldB
    
    f.close()
    
    
    ######################
    ##  Transform data  ##
    ######################
    
    energy_dens = .5 * (
        sc.epsilon_0 * (Ex**2 + Ey**2 + Ez**2)
        + (Bx**2 + By**2 + Bz**2) / sc.mu_0
    )
    
    #################
    ##  Plot data  ##
    #################
    
    figure(figsize=(16,6))
    
    subplot(1,1,1)
    imshow(sum(energy_dens, axis=2), cmap = mpl.cm.cubehelix_r
           , origin='lower', interpolation='none', aspect='auto'
           #, norm=matplotlib.colors.LogNorm(vmin=1.e9)
    )
    
    colorbar()
    
    title("energy density integrated along x \n@t = %iDt"%(t_i))
    
    xlabel("y [cells]")
    ylabel("z [cells]")
    
    show()
    
    close()


# Adios2 File content

In [ ]:
## keys(), get(), item()(?) are built in function of dictionaries

testf = adios2.open(files[0], "r")

testf_variables = testf.available_variables()
print( "VARIABLES:", len(testf_variables), '\n' )

#print(testf_variables)
print("available values:", testf_variables.get("/data/0/fields/B/x") , '\n')

for key, val in testf_variables.items():
    print(key, ":", val['Shape'])

print("="*60)


testf_attributes = testf.available_attributes()
print( "ATTRIBUTES:", len(testf_attributes), '\n' )

print("available values:", testf_attributes.get("/author") , '\n')

for key, val in testf_attributes.items():
    print("%s: %s (%s)"%(key, val['Value'], val['Type']))

print("="*60)

testf.close()